# Launch post-run analysis jobs
Code to create a batch script for launching jobs on cori
Sep 1, 2020

In [2]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [3]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan


In [4]:
val_files={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
          '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
}

In [5]:
results_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'

In [6]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/


20201002_064327/  20201013_151030/  20201014_055904/
20201002_073628/  20201013_211359/  20201015_072548/


In [7]:
# glob.glob(results_loc+'*')

In [8]:
ip_folder='20201015_072548'

In [9]:
### Default dictionary
img_size=128
dict_pars={'job_name':'analysis_128_withspec', 'queue':'debug',
           'time':'00:25:00'}

dict_pars['val_file']=val_files[str(img_size)]
dict_pars['ip_folder']=results_loc+ip_folder

print(dict_pars)

{'job_name': 'analysis_128_withspec', 'queue': 'debug', 'time': '00:25:00', 'val_file': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'ip_folder': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548'}


In [10]:
assert os.path.isdir(dict_pars['ip_folder'])

In [11]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
conda activate v_py3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code'

python $code_dir/2_post_analysis_pandas.py -f {ip_folder}  -v {val_file}
conda deactivate
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [12]:
# bash_strg

In [13]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'
fname='batch_analysis.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_analysis.sh


In [14]:
### Move to staging locations in project space:
os.chdir(staging_loc)

In [15]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=debug
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=00:25:00
#SBATCH --job-name=analysis_128_withspec

echo "--start date" `date` `date +%s`
conda activate v_py3
code_dir='/global/homes/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code'

python $code_dir/2_post_analysis_pandas.py -f /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548  -v /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy
conda deactivate
echo "--end date" `date` `date +%s`


### Submit jobs to cori queue

In [16]:
%%bash -s "$filename" ## Use python variable in bash
sbatch $1

Submitted batch job 35151600
